# NLP Project Pt. 4: Topic Modeling

In [1]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth',250)

corpus_df = pd.read_pickle('corpus_df_spacy.pkl')
corpus_df.head()

,URL,TITLE,AUTHOR,DATE,TEXT,WORDCOUNT,TEXT_NOUNS,TEXT_NOUNS_ADJS,TEXT_NOUNS_VERBS,TEXT_NOUNS_VERBS_ADJS,TEXT_ENTS
0,https://www.newyorker.com/magazine/2022/02/14/annunciation,ANNUNCIATION,LAUREN GROFF,"February 7, 2022","['Some nights, in my dreams, I find myself running through those hills above Palo Alto again. It is always just before dawn, and as I run I smell the sun-crisped fields, the sage, the eucalyptus. The mist falls in starched sheets over the distant...",9373,night dream hill dawn sun field sage eucalyptus mist sheet hill footstep breath peloton cyclist morning fog neighborhood river asphalt road oak grace block time eye pool house moss bougainvillea fern year life parent college graduation dozen carn...,night dream hill dawn sun field sage eucalyptus mist starched sheet distant hill footstep breath peloton cyclist morning fog quiet wealthy neighborhood black river asphalt road great strong armed oak grace block time eye converted pool house moss...,night dream find run hill dawn run smell sun crisp field sage eucalyptus mist fall sheet hill press hear footstep breath peloton cyclist whir morning fog swallow descend neighborhood river asphalt road flatten fly oak spread grace block time awak...,night dream find run hill dawn run smell sun crisp field sage eucalyptus mist fall starched sheet distant hill press hear footstep breath peloton cyclist whir morning fog swallow descend quiet wealthy neighborhood black river asphalt road flatten...,"[Palo Alto, Bay, Mountain View, New England, San Francisco, Chinatown, San Francisco, Redwood City, Mountain View, Titania, Germany, Feuerzangenbowle, Redwood City, Mountain View, New York, Caribbean, the British Virgin Islands, Redwood City, Red..."
1,https://www.newyorker.com/magazine/2022/02/07/once-removed,ONCE REMOVED,ALEXANDER MACLEOD,"January 31, 2022","['She did not want to visit the old lady.', 'Amy studied the stroller, then the bags, then her boyfriend and the baby. She checked her phone: 11:26a.m. It was time to go. Ninety degrees, ninety-per-cent humidity, and, according to Google, more th...",7778,lady stroller bag boyfriend baby phone time degree cent humidity hour way stage icon event minute end min min bus min walk min effort afternoon abort mission baby phone number furnace shoulder humming trick nap eye breathing rivulet drool spine p...,old lady stroller bag boyfriend baby phone time degree cent humidity hour way stage icon olympic event separate minute end min min bus min walk min worth effort hot afternoon abort mission sorry baby right phone number furnace shoulder humming tr...,want visit lady study stroller bag boyfriend baby check phone time degree cent humidity accord hour way stage icon event minute break total end walk min train min bus min walk min effort afternoon abort mission abort baby phone number furnace h...,want visit old lady study stroller bag boyfriend baby check phone time degree cent humidity accord hour way stage icon olympic event separate minute break total end walk min train min bus min walk min worth effort hot afternoon abort mission abor...,"[Inverness County, Nova Scotia, Ontario, Toronto, Montreal, Cape Breton, Turkey, Niagara Falls, Montreal]"
2,https://www.newyorker.com/magazine/2022/01/31/long-distance,LONG DISTANCE,AYSEGUL SAVAS,"January 24, 2022","['Lea changed the sheets when she got up. She’d bought flowers the previous day, tulips that she’d put on the dresser. There were carnations on the kitchen table, in a squat glass vase. She thought they looked cheerful, and not too fussy.', 'The ...",4866,sheet flower day tulip dresser carnation kitchen table glass vase fridge thing olive jam prosciutto cheese wine beer cookie bread cracker café place mind scene bed people mess plate idea indulgence sheet light hour imagination afternoon reason be...,sheet flower previous day tulip dresser carnation kitchen table squat glass vase cheerful fussy fridge thing able olive jam prosciutto cheese wine beer cookie bread round taralli cr

# Vectorization

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF

#function from course notebooks

def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print([[feature_names[i], topic[i].round(3)] for i in topic.argsort()[:-no_top_words - 1:-1]])
    print("\n")
    return model, feature_names, no_top_words

## NMF

In [3]:
#NMF with CV on original text
docs = corpus_df.TEXT
cv = CountVectorizer(stop_words='english', min_df=0.05, max_df=0.6, ngram_range=(1,2))
doc_term = cv.fit_transform(docs)
nmf = NMF(7)
nmf.fit(doc_term)

output = display_topics(nmf, cv.get_feature_names(), 20)

/Users/ami/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  1
[['parents', 3.23], ['phone', 3.029], ['town', 2.498], ['son', 2.327], ['city', 2.309], ['road', 2.25], ['story', 2.245], ['husband', 2.17], ['watched', 2.007], ['apartment', 2.005], ['daughter', 1.966], ['boys', 1.956], ['job', 1.87], ['doesn', 1.859], ['baby', 1.835], ['girls', 1.828], ['sleep', 1.816], ['wall', 1.812], ['dinner', 1.787], ['book', 1.786]]

Topic  2
[['says', 23.031], ['doesn', 2.497], ['asks', 2.18], ['looks', 2.169], ['thinks', 1.516], ['takes', 1.434], ['goes', 1.413], ['comes', 1.236], ['knows', 1.196], ['turns', 1.158], ['feels', 1.072], ['makes', 1.052], ['mark', 0.996], ['gets', 0.952], ['isn', 0.941], ['tells', 0.898], ['husband', 0.854], ['wants', 0.848], ['sees', 0.772], ['guy', 0.751]]

Topic  3
[['jack', 15.358], ['olive', 3.808], ['dog', 2.704], ['phone', 1.155], ['road', 0.946], ['kids', 0.678], ['corn', 0.633], ['farmer', 0.59], ['mom', 0.584], ['girls', 0.576], ['son', 0.574], ['ain', 0.566], ['yeah', 0.487], ['sky', 0.484], ['shit', 0.484], 

In [4]:
#trying NMF with TF-IDF on original text with bigrams
docs = corpus_df.TEXT
tf = TfidfVectorizer(stop_words='english', min_df = 0.05, max_df=0.60, ngram_range=(1,2))
doc_term = tf.fit_transform(docs)
nmf = NMF(7)
nmf.fit(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 20)

/Users/ami/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  1
[['husband', 0.341], ['parents', 0.33], ['daughter', 0.243], ['apartment', 0.239], ['story', 0.237], ['phone', 0.203], ['book', 0.196], ['son', 0.18], ['class', 0.174], ['married', 0.173], ['city', 0.166], ['evening', 0.165], ['teacher', 0.163], ['marriage', 0.147], ['books', 0.146], ['dinner', 0.145], ['reading', 0.143], ['train', 0.141], ['spoke', 0.141], ['girls', 0.141]]

Topic  2
[['says', 1.896], ['asks', 0.357], ['doesn', 0.27], ['looks', 0.244], ['thinks', 0.225], ['takes', 0.191], ['feels', 0.186], ['goes', 0.178], ['tells', 0.162], ['turns', 0.161], ['comes', 0.154], ['knows', 0.148], ['sees', 0.144], ['makes', 0.13], ['sits', 0.126], ['wants', 0.124], ['puts', 0.124], ['gets', 0.117], ['starts', 0.117], ['stands', 0.106]]

Topic  3
[['james', 1.702], ['doctor', 0.065], ['ship', 0.056], ['daughter', 0.052], ['mary', 0.049], ['son', 0.044], ['motel', 0.039], ['ain', 0.039], ['willing', 0.038], ['dog', 0.038], ['mr', 0.038], ['yeah', 0.037], ['maid', 0.036], ['book', 

In [5]:
#best topics

#NMF + TF-IDF + nouns only

docs = corpus_df.TEXT_NOUNS
tf = TfidfVectorizer(stop_words='english', min_df = 0.03, max_df=0.55)
doc_term = tf.fit_transform(docs)
nmf = NMF(n_components=6, init='nndsvda', max_iter=450)
nmf.fit(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 20)


Topic  1
[['apartment', 0.424], ['student', 0.315], ['class', 0.298], ['train', 0.273], ['office', 0.27], ['hotel', 0.268], ['bar', 0.263], ['painting', 0.255], ['building', 0.241], ['dinner', 0.241], ['writer', 0.238], ['coffee', 0.235], ['restaurant', 0.233], ['party', 0.233], ['city', 0.232], ['drink', 0.22], ['desk', 0.22], ['teacher', 0.22], ['movie', 0.218], ['film', 0.214]]

Topic  2
[['road', 0.46], ['river', 0.268], ['wind', 0.265], ['stone', 0.264], ['sun', 0.26], ['field', 0.258], ['sea', 0.252], ['horse', 0.24], ['grass', 0.23], ['sky', 0.226], ['wood', 0.215], ['beach', 0.214], ['boat', 0.204], ['rain', 0.199], ['ground', 0.196], ['truck', 0.195], ['lake', 0.193], ['bird', 0.191], ['rock', 0.191], ['mountain', 0.19]]

Topic  3
[['kid', 0.863], ['guy', 0.818], ['mom', 0.462], ['shit', 0.266], ['dad', 0.265], ['cop', 0.26], ['brother', 0.256], ['ass', 0.225], ['yard', 0.18], ['son', 0.179], ['gun', 0.169], ['movie', 0.161], ['fuck', 0.154], ['stuff', 0.144], ['hell', 0.131]

In [6]:
#NMF + TF-IDF + nouns + adjs only
docs = corpus_df.TEXT_NOUNS_ADJS
tf = TfidfVectorizer(stop_words='english', min_df = 0.05, max_df=0.60)
doc_term = tf.fit_transform(docs)
nmf = NMF(n_components=5, init='nndsvda', max_iter=450)
nmf.fit(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 20)


Topic  1
[['parent', 0.381], ['apartment', 0.317], ['student', 0.311], ['husband', 0.308], ['class', 0.274], ['phone', 0.267], ['love', 0.254], ['daughter', 0.25], ['teacher', 0.237], ['office', 0.236], ['painting', 0.218], ['dinner', 0.217], ['city', 0.216], ['evening', 0.215], ['train', 0.21], ['party', 0.209], ['marriage', 0.208], ['college', 0.207], ['letter', 0.204], ['hotel', 0.204]]

Topic  2
[['road', 0.389], ['wind', 0.244], ['stone', 0.235], ['sun', 0.234], ['river', 0.233], ['field', 0.227], ['town', 0.223], ['village', 0.214], ['sea', 0.21], ['horse', 0.209], ['sky', 0.205], ['grass', 0.2], ['wood', 0.192], ['rain', 0.191], ['beach', 0.179], ['ground', 0.178], ['boat', 0.177], ['green', 0.176], ['truck', 0.171], ['bird', 0.17]]

Topic  3
[['kid', 0.75], ['guy', 0.743], ['mom', 0.401], ['fucking', 0.274], ['phone', 0.273], ['shit', 0.262], ['brother', 0.248], ['dad', 0.237], ['cop', 0.229], ['ass', 0.22], ['nice', 0.193], ['movie', 0.177], ['fuck', 0.167], ['bag', 0.162], [

In [7]:
#NMF + TF-IDF + nouns + verbs only
docs = corpus_df.TEXT_NOUNS_VERBS
tf = TfidfVectorizer(stop_words='english', min_df = 0.05, max_df=0.60)
doc_term = tf.fit_transform(docs)
nmf = NMF(n_components=5, init='nndsvda', max_iter=450)
nmf.fit(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 20)


Topic  1
[['parent', 0.394], ['student', 0.307], ['husband', 0.305], ['apartment', 0.303], ['class', 0.261], ['daughter', 0.26], ['teacher', 0.249], ['office', 0.224], ['marry', 0.216], ['marriage', 0.214], ['letter', 0.213], ['evening', 0.212], ['train', 0.211], ['painting', 0.211], ['city', 0.209], ['college', 0.204], ['dinner', 0.203], ['writer', 0.202], ['restaurant', 0.194], ['party', 0.194]]

Topic  2
[['road', 0.369], ['wind', 0.24], ['sun', 0.23], ['stone', 0.228], ['river', 0.219], ['sea', 0.21], ['sky', 0.209], ['field', 0.209], ['rain', 0.201], ['town', 0.198], ['rise', 0.19], ['grass', 0.188], ['horse', 0.182], ['wood', 0.179], ['village', 0.177], ['beach', 0.177], ['lift', 0.172], ['boat', 0.171], ['climb', 0.168], ['ground', 0.168]]

Topic  3
[['kid', 0.742], ['guy', 0.715], ['mom', 0.389], ['fuck', 0.33], ['shit', 0.269], ['brother', 0.265], ['dad', 0.232], ['cop', 0.224], ['ass', 0.213], ['hit', 0.191], ['movie', 0.187], ['kill', 0.165], ['bag', 0.155], ['stuff', 0.151

In [8]:
#NMF + TF-IDF + nouns + verbs + adjs
docs = corpus_df.TEXT_NOUNS_VERBS_ADJS
tf = TfidfVectorizer(stop_words='english', min_df = 0.02, max_df=0.55)
doc_term = tf.fit_transform(docs) 
nmf = NMF(n_components=6, init='nndsvda', max_iter=450)
NMF_doc_topic = nmf.fit_transform(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 25)


Topic  1
[['student', 0.298], ['husband', 0.275], ['daughter', 0.249], ['teacher', 0.24], ['class', 0.23], ['writer', 0.209], ['marriage', 0.197], ['son', 0.183], ['letter', 0.18], ['marry', 0.175], ['college', 0.171], ['novel', 0.168], ['painting', 0.164], ['city', 0.161], ['poem', 0.159], ['certain', 0.154], ['office', 0.154], ['conversation', 0.153], ['art', 0.153], ['study', 0.151], ['reply', 0.15], ['train', 0.149], ['death', 0.149], ['apartment', 0.146], ['wish', 0.145]]

Topic  2
[['bus', 0.23], ['box', 0.207], ['bottle', 0.201], ['bedroom', 0.198], ['tea', 0.191], ['coat', 0.191], ['kiss', 0.186], ['apartment', 0.182], ['bathroom', 0.18], ['coffee', 0.176], ['wine', 0.168], ['bar', 0.165], ['cat', 0.162], ['mirror', 0.162], ['plate', 0.157], ['rain', 0.157], ['sofa', 0.155], ['hall', 0.152], ['jacket', 0.147], ['paint', 0.145], ['desk', 0.143], ['lift', 0.141], ['towel', 0.141], ['shop', 0.14], ['shoe', 0.137]]

Topic  3
[['guy', 0.653], ['mom', 0.359], ['fuck', 0.331], ['brot

In [9]:
#looking at topics for each doc
NMF_doc_topic = nmf.fit_transform(doc_term)

NMF_doc_topic_df = pd.DataFrame(NMF_doc_topic.round(3),
                 columns = [ix for ix, val in enumerate(nmf.components_)]
                )
NMF_doc_topic_df

,0,1,2,3,4,5
0,0.043,0.096,0.013,0.058,0.077,0.045
1,0.005,0.090,0.020,0.182,0.000,0.000
2,0.094,0.099,0.012,0.000,0.000,0.000
3,0.101,0.009,0.078,0.000,0.016,0.027
4,0.049,0.031,0.000,0.054,0.015,0.005
...,...,...,...,...,...,...
939,0.035,0.058,0.002,0.035,0.008,0.060
940,0.037,0.120,0.070,0.005,0.000,0.000
941,0.037,0.172,0.009,0.026,0.000,0.007
942,0.000,0.000,0.005,0.136,0.000,0.094


In [10]:
doc_term_df = pd.DataFrame(doc_term.toarray(), columns=tf.get_feature_names())
doc_term_df.head()

,abandon,abandonment,abdomen,abduct,abide,ability,abortion,abrupt,absence,absent,...,youth,youthful,zigzag,zip,zipper,zombie,zone,zoo,zoom,élite
0,0.0,0.0,0.0,0.0,0.0,0.034527,0.0,0.000000,0.000000,0.020317,...,0.031073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.039368,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.035504,0.000000,...,0.034431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.031750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#looking at top doc for each topic
for ix, val in enumerate(nmf.components_):
    max_story = np.argmax(NMF_doc_topic_df[ix])
    print('Topic', ix+1, max_story, corpus_df.TITLE[max_story], corpus_df.AUTHOR[max_story])

Topic 1 779 THE ALBANIAN WRITERS' UNION AS MIRRORED BY A WOMAN ISMAIL KADARE
Topic 2 486 SWEET DREAMS PETER STAMM
Topic 3 22 THE MOM OF BOLD ACTION GEORGE SAUNDERS
Topic 4 58 THE WINGED THING PATRICIA LOCKWOOD
Topic 5 555 THE YELLOW SAMANTHA HUNT
Topic 6 490 TRANSATLANTIC COLUM MCCANN


In [12]:
#looking at top terms for Sunrise, Sunset
(doc_term_df.loc[213].sort_values(ascending=False)).head(20)

husband      0.298740
son          0.297999
terrace      0.295327
grandson     0.269114
daughter     0.267363
doll         0.250222
baby         0.174818
gurney       0.127792
toast        0.111103
whistle      0.093893
church       0.091348
birth        0.091123
doctor       0.090895
pool         0.089675
oil          0.084820
strap        0.078126
champagne    0.077757
devil        0.077366
lower        0.077176
wrap         0.076927
Name: 213, dtype: float64

In [13]:
corpus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   URL                    944 non-null    object
 1   TITLE                  944 non-null    object
 2   AUTHOR                 944 non-null    object
 3   DATE                   944 non-null    object
 4   TEXT                   944 non-null    object
 5   WORDCOUNT              944 non-null    int64 
 6   TEXT_NOUNS             944 non-null    object
 7   TEXT_NOUNS_ADJS        944 non-null    object
 8   TEXT_NOUNS_VERBS       944 non-null    object
 9   TEXT_NOUNS_VERBS_ADJS  944 non-null    object
 10  TEXT_ENTS              944 non-null    object
dtypes: int64(1), object(10)
memory usage: 81.2+ KB


In [14]:
NMF_doc_topic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       944 non-null    float64
 1   1       944 non-null    float64
 2   2       944 non-null    float64
 3   3       944 non-null    float64
 4   4       944 non-null    float64
 5   5       944 non-null    float64
dtypes: float64(6)
memory usage: 44.4 KB


In [15]:
NMF_doc_topic_df.loc[490]

0    0.000
1    0.044
2    0.000
3    0.000
4    0.000
5    0.199
Name: 490, dtype: float64

In [52]:
#best topics

#NMF + TF-IDF + nouns only

docs = corpus_df.TEXT_NOUNS
tf = TfidfVectorizer(stop_words='english', min_df = 0.03, max_df=0.55)
doc_term = tf.fit_transform(docs)
nmf = NMF(n_components=6, init='nndsvda', max_iter=450)

NMF_doc_topic = nmf.fit_transform(doc_term)

NMF_doc_topic_df = pd.DataFrame(NMF_doc_topic.round(3),
                 columns = [ix for ix, val in enumerate(nmf.components_)]
                )
NMF_doc_topic_df

,0,1,2,3,4,5
0,0.047,0.064,0.011,0.023,0.088,0.093
1,0.044,0.022,0.012,0.259,0.000,0.013
2,0.131,0.018,0.000,0.000,0.001,0.026
3,0.113,0.025,0.039,0.000,0.013,0.020
4,0.011,0.010,0.000,0.000,0.017,0.151
...,...,...,...,...,...,...
939,0.030,0.074,0.019,0.038,0.015,0.046
940,0.065,0.038,0.063,0.018,0.000,0.007
941,0.099,0.049,0.000,0.033,0.000,0.055
942,0.000,0.062,0.000,0.062,0.001,0.097


In [53]:
#pickle
NMF_doc_topic_df.to_pickle("NMF_doc_topic_df3.pkl")

## LSA 
***

In [17]:
#LSA with TF-IDF
#looking at nouns only
docs = corpus_df.TEXT_NOUNS
tf = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.55)
doc_term = tf.fit_transform(docs)
lsa = TruncatedSVD(5)
lsa.fit(doc_term)
topic_term = lsa.components_
output = display_topics(model=lsa, feature_names=tf.get_feature_names(), no_top_words=20)


Topic  1
[['dog', 0.109], ['husband', 0.107], ['son', 0.107], ['kid', 0.107], ['baby', 0.105], ['guy', 0.098], ['brother', 0.093], ['daughter', 0.092], ['apartment', 0.089], ['sister', 0.085], ['road', 0.08], ['doctor', 0.077], ['bar', 0.07], ['city', 0.07], ['dinner', 0.069], ['coffee', 0.067], ['office', 0.064], ['box', 0.063], ['bus', 0.063], ['party', 0.062]]

Topic  2
[['dog', 0.389], ['road', 0.192], ['river', 0.109], ['gun', 0.104], ['yard', 0.101], ['horse', 0.101], ['field', 0.094], ['truck', 0.088], ['rock', 0.088], ['wind', 0.085], ['sun', 0.084], ['grass', 0.082], ['kid', 0.082], ['wood', 0.08], ['bird', 0.076], ['fence', 0.072], ['stone', 0.071], ['cabin', 0.071], ['porch', 0.069], ['cow', 0.068]]

Topic  3
[['baby', 0.421], ['kid', 0.352], ['guy', 0.294], ['mom', 0.209], ['dog', 0.167], ['dad', 0.103], ['cop', 0.099], ['shit', 0.098], ['sister', 0.091], ['ass', 0.089], ['doctor', 0.089], ['nurse', 0.076], ['apartment', 0.074], ['movie', 0.068], ['couch', 0.066], ['dollar

In [18]:
#nouns plus adjectives
docs = corpus_df.TEXT_NOUNS_ADJS
tf = TfidfVectorizer(stop_words='english', max_df=0.55)
doc_term = tf.fit_transform(docs)
lsa = TruncatedSVD(7)
lsa.fit(doc_term)
topic_term = lsa.components_
output = display_topics(model=lsa, feature_names=tf.get_feature_names(), no_top_words=20)


Topic  1
[['dog', 0.097], ['baby', 0.095], ['husband', 0.095], ['kid', 0.093], ['son', 0.092], ['brother', 0.084], ['guy', 0.082], ['apartment', 0.079], ['daughter', 0.079], ['sister', 0.076], ['road', 0.072], ['doctor', 0.068], ['dinner', 0.061], ['city', 0.06], ['bar', 0.06], ['coffee', 0.058], ['bus', 0.056], ['box', 0.056], ['office', 0.055], ['bottle', 0.055]]

Topic  2
[['dog', 0.386], ['road', 0.16], ['river', 0.093], ['yard', 0.086], ['horse', 0.084], ['gun', 0.083], ['kid', 0.081], ['rock', 0.08], ['field', 0.079], ['sun', 0.075], ['wind', 0.074], ['truck', 0.074], ['cabin', 0.074], ['wood', 0.072], ['porch', 0.069], ['grass', 0.067], ['stone', 0.065], ['boat', 0.064], ['fish', 0.064], ['fence', 0.061]]

Topic  3
[['baby', 0.347], ['kid', 0.31], ['guy', 0.263], ['mom', 0.193], ['dog', 0.12], ['cop', 0.095], ['shit', 0.092], ['dad', 0.088], ['apartment', 0.087], ['fucking', 0.079], ['ass', 0.079], ['brother', 0.068], ['doctor', 0.067], ['movie', 0.065], ['nurse', 0.064], ['son

In [19]:
#trying nouns and verbs
docs = corpus_df.TEXT_NOUNS_VERBS
tf = TfidfVectorizer(stop_words='english', max_df=0.55)
doc_term = tf.fit_transform(docs)
lsa = TruncatedSVD(7)
lsa.fit(doc_term)
topic_term = lsa.components_
output = display_topics(model=lsa, feature_names=tf.get_feature_names(), no_top_words=20)


Topic  1
[['dog', 0.093], ['husband', 0.091], ['baby', 0.089], ['son', 0.089], ['brother', 0.081], ['guy', 0.078], ['apartment', 0.076], ['daughter', 0.075], ['sister', 0.072], ['road', 0.069], ['doctor', 0.065], ['bar', 0.059], ['city', 0.059], ['dinner', 0.058], ['coffee', 0.056], ['kiss', 0.056], ['bus', 0.054], ['box', 0.054], ['dream', 0.053], ['office', 0.053]]

Topic  2
[['dog', 0.311], ['road', 0.145], ['river', 0.087], ['gun', 0.08], ['yard', 0.077], ['sun', 0.074], ['rock', 0.072], ['horse', 0.071], ['fish', 0.07], ['truck', 0.07], ['field', 0.07], ['wind', 0.068], ['rain', 0.065], ['wood', 0.064], ['cabin', 0.063], ['boat', 0.061], ['grass', 0.061], ['porch', 0.059], ['stone', 0.057], ['fence', 0.056]]

Topic  3
[['baby', 0.442], ['guy', 0.195], ['mom', 0.174], ['husband', 0.119], ['doctor', 0.107], ['apartment', 0.104], ['nurse', 0.099], ['sister', 0.094], ['daughter', 0.094], ['son', 0.089], ['fuck', 0.089], ['shit', 0.082], ['brother', 0.079], ['dad', 0.077], ['hospital'

In [20]:
#LSA with nouns, verbs, adjectives
docs = corpus_df.TEXT_NOUNS_VERBS_ADJS
tf = TfidfVectorizer(stop_words='english', max_df=0.55)
doc_term = tf.fit_transform(docs)
lsa = TruncatedSVD(7)
lsa.fit(doc_term)
topic_term = lsa.components_
output = display_topics(model=lsa, feature_names=tf.get_feature_names(), no_top_words=20)


Topic  1
[['dog', 0.086], ['baby', 0.084], ['husband', 0.083], ['son', 0.082], ['brother', 0.074], ['guy', 0.072], ['apartment', 0.069], ['daughter', 0.069], ['sister', 0.067], ['road', 0.064], ['doctor', 0.06], ['bar', 0.054], ['city', 0.053], ['dinner', 0.053], ['coffee', 0.052], ['kiss', 0.051], ['bus', 0.05], ['box', 0.05], ['dream', 0.049], ['office', 0.049]]

Topic  2
[['dog', 0.279], ['road', 0.135], ['river', 0.081], ['gun', 0.076], ['sun', 0.073], ['yard', 0.071], ['rock', 0.07], ['truck', 0.069], ['fish', 0.068], ['wind', 0.068], ['horse', 0.067], ['field', 0.067], ['rain', 0.064], ['cabin', 0.063], ['wood', 0.062], ['boat', 0.061], ['baby', 0.059], ['grass', 0.059], ['stone', 0.055], ['porch', 0.054]]

Topic  3
[['baby', 0.349], ['guy', 0.236], ['mom', 0.18], ['fuck', 0.114], ['apartment', 0.112], ['shit', 0.098], ['fucking', 0.092], ['brother', 0.09], ['dad', 0.085], ['cop', 0.083], ['ass', 0.081], ['doctor', 0.079], ['nurse', 0.076], ['couch', 0.073], ['son', 0.072], ['si

In [21]:
doc_topic = lsa.fit_transform(doc_term)

doc_topic_df = pd.DataFrame(doc_topic.round(3),
                 index = corpus_df.TITLE,
                 columns = [ix for ix, val in enumerate(lsa.components_)]
                )
doc_topic_df

,0,1,2,3,4,5,6
TITLE,,,,,,,
ANNUNCIATION,0.362,0.040,-0.023,0.039,0.108,0.031,-0.013
ONCE REMOVED,0.318,0.015,0.137,0.162,-0.013,0.090,0.103
LONG DISTANCE,0.262,-0.109,-0.027,-0.034,0.024,0.084,-0.032
"WHAT'S THE DEAL, HUMMINGBIRD?",0.272,-0.081,0.014,-0.101,-0.022,-0.035,0.028
FIREWORKS,0.184,-0.047,-0.005,0.069,0.044,-0.022,-0.031
...,...,...,...,...,...,...,...
THE GUARDIANS,0.252,0.023,-0.049,0.051,-0.007,-0.001,0.014
U.F.O. IN KUSHIRO,0.266,-0.023,0.022,-0.039,-0.028,0.093,-0.072
NOBODY'S BUSINESS,0.338,-0.032,-0.008,0.043,-0.016,0.137,-0.074


In [22]:
#looking at top doc for each topic
for ix, val in enumerate(lsa.components_):
    max_story = np.argmax(doc_topic_df[ix], axis=-1)
    print(max_story, corpus_df.TITLE[max_story], corpus_df.AUTHOR[max_story], np.max(val))

764 ONCE IN A LIFETIME JHUMPA LAHIRI 0.08594190554048627
516 DOG RUN MOON CALLAN WINK 0.2747004896453223
529 HOME GEORGE SAUNDERS 0.3479108312470661
394 A SHELTERED WOMAN YIYUN LI 0.41269616737029624
695 THE DOG RODDY DOYLE 0.7428194402995678
809 SOLACE DONALD ANTRIM 0.10623639330303336
838 MIRACLE JUDY BUDNITZ 0.41348752836494407


## LDA
***

In [23]:
from gensim import corpora, models, matutils

#looking at nouns only
docs = corpus_df.TEXT_NOUNS
vectorizer = CountVectorizer(stop_words='english', max_df=0.55)
term_doc_matrix = vectorizer.fit_transform(docs).transpose()

corpus = matutils.Sparse2Corpus(term_doc_matrix)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=7, id2word=id2word, passes=150)

lda.print_topics()

[(0,
  '0.006*"cat" + 0.005*"husband" + 0.004*"bottle" + 0.004*"box" + 0.004*"bedroom" + 0.004*"sister" + 0.004*"son" + 0.004*"skin" + 0.003*"dinner" + 0.003*"coffee"'),
 (1,
  '0.023*"fish" + 0.022*"sister" + 0.008*"cow" + 0.006*"niece" + 0.005*"boat" + 0.005*"milk" + 0.005*"nest" + 0.004*"cat" + 0.004*"gangster" + 0.004*"uncle"'),
 (2,
  '0.012*"kid" + 0.010*"guy" + 0.009*"baby" + 0.005*"mom" + 0.005*"brother" + 0.005*"son" + 0.004*"apartment" + 0.003*"dollar" + 0.003*"bus" + 0.003*"doctor"'),
 (3,
  '0.006*"husband" + 0.004*"student" + 0.004*"daughter" + 0.004*"class" + 0.003*"teacher" + 0.003*"son" + 0.003*"city" + 0.003*"conversation" + 0.003*"death" + 0.003*"apartment"'),
 (4,
  '0.007*"road" + 0.003*"summer" + 0.003*"bird" + 0.003*"monkey" + 0.003*"field" + 0.003*"rain" + 0.003*"tea" + 0.003*"bar" + 0.002*"wind" + 0.002*"train"'),
 (5,
  '0.006*"road" + 0.005*"sun" + 0.005*"sea" + 0.004*"sky" + 0.004*"stone" + 0.004*"gun" + 0.004*"wind" + 0.004*"ground" + 0.004*"beach" + 0.003*"

In [24]:
#nouns and verbs
docs = corpus_df.TEXT_NOUNS_VERBS
vectorizer = CountVectorizer(stop_words='english', min_df = 0.05, max_df=0.60)
term_doc_matrix = vectorizer.fit_transform(docs).transpose()

corpus = matutils.Sparse2Corpus(term_doc_matrix)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=6, id2word=id2word, passes=250)

lda.print_topics()

[(0,
  '0.009*"dog" + 0.006*"stone" + 0.005*"sun" + 0.005*"sea" + 0.005*"wind" + 0.005*"sky" + 0.005*"rise" + 0.005*"bird" + 0.004*"rock" + 0.004*"village"'),
 (1,
  '0.011*"baby" + 0.007*"husband" + 0.007*"sister" + 0.006*"doctor" + 0.005*"bag" + 0.005*"dog" + 0.005*"food" + 0.005*"son" + 0.005*"box" + 0.004*"parent"'),
 (2,
  '0.014*"road" + 0.010*"town" + 0.007*"truck" + 0.006*"gun" + 0.006*"fish" + 0.006*"horse" + 0.005*"yard" + 0.005*"ground" + 0.005*"field" + 0.004*"driver"'),
 (3,
  '0.006*"parent" + 0.006*"husband" + 0.006*"son" + 0.006*"daughter" + 0.004*"student" + 0.004*"class" + 0.004*"learn" + 0.004*"death" + 0.004*"teacher" + 0.004*"marry"'),
 (4,
  '0.018*"kid" + 0.017*"guy" + 0.007*"fuck" + 0.007*"mom" + 0.006*"movie" + 0.005*"hit" + 0.005*"shit" + 0.005*"dad" + 0.005*"bar" + 0.005*"brother"'),
 (5,
  '0.005*"painting" + 0.004*"hotel" + 0.004*"train" + 0.004*"paint" + 0.004*"evening" + 0.004*"bar" + 0.003*"mirror" + 0.003*"coffee" + 0.003*"apartment" + 0.003*"clothe"')]

In [25]:
#nouns and verbs and adjs
docs = corpus_df.TEXT_NOUNS_VERBS_ADJS
vectorizer = CountVectorizer(stop_words='english', min_df = 10, max_df=0.60)
term_doc_matrix = vectorizer.fit_transform(docs).transpose()

corpus = matutils.Sparse2Corpus(term_doc_matrix)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=6, id2word=id2word, passes=350)

lda.print_topics()

[(0,
  '0.007*"dog" + 0.006*"road" + 0.003*"wind" + 0.003*"town" + 0.003*"sun" + 0.003*"yard" + 0.003*"truck" + 0.003*"ground" + 0.003*"field" + 0.003*"cat"'),
 (1,
  '0.003*"bedroom" + 0.003*"clothe" + 0.003*"parent" + 0.003*"bag" + 0.003*"evening" + 0.002*"bottle" + 0.002*"kiss" + 0.002*"tea" + 0.002*"husband" + 0.002*"press"'),
 (2,
  '0.004*"human" + 0.003*"beach" + 0.003*"sea" + 0.003*"hotel" + 0.002*"death" + 0.002*"sky" + 0.002*"fear" + 0.002*"space" + 0.002*"form" + 0.002*"certain"'),
 (3,
  '0.008*"apartment" + 0.006*"husband" + 0.005*"parent" + 0.004*"daughter" + 0.004*"student" + 0.004*"class" + 0.004*"building" + 0.004*"office" + 0.004*"movie" + 0.004*"bar"'),
 (4,
  '0.011*"kid" + 0.008*"baby" + 0.007*"guy" + 0.005*"sister" + 0.004*"brother" + 0.004*"mom" + 0.004*"fuck" + 0.004*"doctor" + 0.004*"son" + 0.003*"bag"'),
 (5,
  '0.004*"son" + 0.003*"town" + 0.003*"parent" + 0.003*"learn" + 0.002*"death" + 0.002*"grandfather" + 0.002*"country" + 0.002*"village" + 0.002*"letter"

In [27]:
# create dataframe showing topic probability scores for each document

from collections import defaultdict

def lda_df_maker(data, n_topics):
    topic_doc_dist = defaultdict(list)
    for ix, val in enumerate(data):
        doc_top_list = lda.get_document_topics(corpus[ix], minimum_probability=0, minimum_phi_value=0)
        for pair in doc_top_list:
            topic_num = pair[0]
            topic_score = pair[1]
            topic_doc_dist[topic_num].append(np.round(topic_score, 3))
    return topic_doc_dist

lda.get_document_topics(corpus[2], minimum_probability=0, minimum_phi_value=0)

[(0, 0.008445222),
 (1, 0.008445222),
 (2, 0.008445222),
 (3, 0.008445222),
 (4, 0.95777386),
 (5, 0.008445222)]

In [34]:
lda_df_dict = lda_df_maker(corpus, n_topics=4)

In [30]:
lda_df = pd.DataFrame(lda_df_dict)
lda_df

,0,1,2,3,4,5
0,0.007,0.007,0.007,0.007,0.965,0.007
1,0.008,0.008,0.008,0.008,0.960,0.008
2,0.008,0.008,0.008,0.008,0.958,0.008
3,0.009,0.009,0.009,0.009,0.957,0.009
4,0.014,0.014,0.014,0.014,0.932,0.014
...,...,...,...,...,...,...
939,0.009,0.009,0.009,0.009,0.956,0.009
940,0.008,0.008,0.008,0.008,0.958,0.008
941,0.008,0.008,0.008,0.008,0.961,0.008
942,0.012,0.012,0.012,0.012,0.938,0.012


In [37]:
dominant_topic = [np.argmax(lda_df.values, axis=1)]
dominant_topic

[array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [38]:
lda_df.insert(0, 'TITLE', corpus_df.TITLE, allow_duplicates=False)
lda_df.insert(1, 'AUTHOR', corpus_df.AUTHOR, allow_duplicates=True)
lda_df

,TITLE,AUTHOR,0,1,2,3,4,5
0,ANNUNCIATION,LAUREN GROFF,0.007,0.007,0.007,0.007,0.965,0.007
1,ONCE REMOVED,ALEXANDER MACLEOD,0.008,0.008,0.008,0.008,0.960,0.008
2,LONG DISTANCE,AYSEGUL SAVAS,0.008,0.008,0.008,0.008,0.958,0.008
3,"WHAT'S THE DEAL, HUMMINGBIRD?",ARTHUR KRYSTAL,0.009,0.009,0.009,0.009,0.957,0.009
4,FIREWORKS,GRAHAM SWIFT,0.014,0.014,0.014,0.014,0.932,0.014
...,...,...,...,...,...,...,...,...
939,THE GUARDIANS,JOHN UPDIKE,0.009,0.009,0.009,0.009,0.956,0.009
940,U.F.O. IN KUSHIRO,HARUKI MURAKAMI,0.008,0.008,0.008,0.008,0.958,0.008
941,NOBODY'S BUSINESS,JHUMPA LAHIRI,0.008,0.008,0.008,0.008,0.961,0.008
942,THE SHAWL,LOUISE ERDRICH,0.012,0.012,0.012,0.012,0.938,0.012


In [40]:
#looking at sally rooney, unread messages
lda.get_document_topics(corpus[27])

[(4, 0.9585572)]

In [42]:
#looking at george saunders, semplica girl diaries
lda.get_document_topics(corpus[462])

[(0, 0.010164557),
 (1, 0.010164559),
 (2, 0.01016455),
 (3, 0.0101645505),
 (4, 0.94917727),
 (5, 0.01016456)]

# CorEx

In [ ]:
#!pip install corextopic
#!pip install networkx

In [45]:
from corextopic import corextopic as ct

docs = corpus_df.TEXT
tf = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.60, ngram_range=(1,2))
tfidf = tf.fit_transform(docs)
vocab = tf.get_feature_names()

model = ct.Corex(n_hidden=5, seed=42)
model = model.fit(tfidf, words=vocab)

topics = model.get_topics()
for topic_n,topic in enumerate(topics):
    # w: word, mi: mutual information, s: sign
    topic = [(w,mi,s) if s > 0 else ('~'+w,mi,s) for w,mi,s in topic]
    # Unpack the info about the topic
    words,mis,signs = zip(*topic)    
    # Print topic
    topic_str = str(topic_n+1)+': '+', '.join(words)
    print(topic_str)

1: says, asks, thinks, looks, feels, takes, sees, doesn, goes, tells
2: internet, eve, backward, leader, plus, current, ~precisely, ~precious, ~pre, ~praying
3: jack, hope, ~relatively, ~released, ~reliable, ~relieved, ~remained, ~reminded, remorse, ~remove
4: abandoned, press, pressed, pretend, pretty, prevent, pride, ~priest, printed, privacy
5: abandon, problem, professional, ~promises, pronounced, proof, propped, prospect, protection, protested


In [46]:
docs = corpus_df.TEXT_NOUNS
tf = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.60, ngram_range=(1,2))
tfidf = tf.fit_transform(docs)
vocab = tf.get_feature_names()

model = ct.Corex(n_hidden=5, seed=42)
model = model.fit(tfidf, words=vocab)

topics = model.get_topics()
for topic_n,topic in enumerate(topics):
    # w: word, mi: mutual information, s: sign
    topic = [(w,mi,s) if s > 0 else ('~'+w,mi,s) for w,mi,s in topic]
    # Unpack the info about the topic
    words,mis,signs = zip(*topic)    
    # Print topic
    topic_str = str(topic_n+1)+': '+', '.join(words)
    print(topic_str)

1: lift, hut, trail, pass, snow, pole, mountain, valley, slope, ridge
2: ~account, ~reading, razor, ray, railing, rage, rag, ~race, rabbit, ~prospect
3: act, ~reflection, ~rearview mirror, rate, ~range, rack, punishment, ~punch, ~pump, public
4: ~accident, ~prize, ~privacy, ~principle, ~presence, prayer, ~power, ~powder, ~posture, pork
5: absence, ~place man, ~pitch, ~pit, pink, ~pin, piece paper, ~pickup, phone, philosophy


In [47]:
docs = corpus_df.TEXT_NOUNS_VERBS_ADJS
tf = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.80, ngram_range=(1,2))
tfidf = tf.fit_transform(docs)
vocab = tf.get_feature_names()

anchors = [
    ["age", "lifetime", "life", "death", "memory"],
    ["COVID", "pandemic", "virus", "plague"],
    ["attraction", "desire", "sex", "romance", "girlfriend", "boyfriend", "wedding", "marriage"],
    ["paycheck", "bill", "estate", "rate", "debit", "credit"],
    ["English", "immigrant", "country"], [], [], []
]

anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=8, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=4 # Tell the model how much it should rely on the anchors
)

topics = model.get_topics()
for topic_n,topic in enumerate(topics):
    # w: word, mi: mutual information, s: sign
    topic = [(w,mi,s) if s > 0 else ('~'+w,mi,s) for w,mi,s in topic]
    # Unpack the info about the topic
    words,mis,signs = zip(*topic)    
    # Print topic
    topic_str = str(topic_n+1)+': '+', '.join(words)
    print(topic_str)

1: freezer, pizza, teacher, english, way feel, chest, ~rainy, ~ragged, ~rage, ~range
2: register, cash, daddy, drone, mall, battery, birthday, ~plague, ~railing, ~quote
3: boat, bridge, arch, skull, bank, river, current, stream, mobile, ~remove
4: frog, lick, pond, tongue, mud, transport, fairy, ex, ~represent, ~reservation
5: internet, poem, leader, poet, poetry, reader, ~absent, ~prison, private, ~professional
6: able, pre, praise, possible, portrait, ~plenty, pleased, ~plaster, plan, ~pipe
7: accompany, ~rattle, ~rat, ~rake, ~raise hand, ~radio, punishment, ~pull, pub, proud
8: ~accident, ~previous, ~preserve, ~present, ~predict, ~powerful, powder, ~pour glass, ~position, ~polite


In [48]:
#try with countvectorizer
from corextopic import corextopic as ct
docs = corpus_df.TEXT_NOUNS_VERBS
vectorizer = CountVectorizer(max_features=20000,
                             stop_words='english', token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True, max_df=0.60)

doc_word = vectorizer.fit_transform(docs)
words = list(np.asarray(vectorizer.get_feature_names()))

anchors = [
    ["age", "lifetime", "life", "death", "memory", "die", "remember", "memorial", "widow"],
    ["COVID", "pandemic", "virus", "plague"],
    ["attraction", "desire", "sex", "romance", "girlfriend", "boyfriend", "wedding", "marriage", "kiss"],
    ["paycheck", "bill", "estate", "rate", "debit", "credit", "pay", "debt"],
    ["English", "immigrant", "country", "immigrate"]
]

anchors = [
    [a for a in topic if a in words]
    for topic in anchors
]


topic_model = ct.Corex(n_hidden=6, words=words, seed=1)
topic_model.fit(doc_word, words=words, docs=docs, anchors=anchors, anchor_strength=4)

topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: death,memory,lifetime,widow,discover,fear,past,presence,form,spirit
1: plague,virus,pandemic,scan,freak,vacation,effect,bullshit,garbage,image
2: sex,marriage,girlfriend,desire,boyfriend,romance,wedding,kiss,attraction,bathroom
3: rate,estate,debt,credit,problem,site,deal,project,plan,dollar
4: country,roll,nose,rise,grip,breath,neck,wave,squeeze,ground
5: ivy,mouthpiece,trailer,craze,anesthetic,gypsy,hoodie,litre,bacon,irregularity


In [51]:
#try with countvectorizer, no anchors
from corextopic import corextopic as ct

vectorizer = CountVectorizer(max_features=20000,
                             stop_words='english', token_pattern="\\b[a-z][a-z]+\\b",
                             max_df=0.60, binary=True)

doc_word = vectorizer.fit_transform(docs)
words = list(np.asarray(vectorizer.get_feature_names()))

topic_model = ct.Corex(n_hidden=6, words=words, seed=1)
topic_model.fit(doc_word, words=words, docs=docs)

topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: roll,breath,sky,wave,ground,palm,tooth,rise,sink,drag
1: self,office,offer,check,college,problem,result,lack,game,coffee
2: childhood,manner,engage,teacher,folder,spoil,script,calendar,overlook,dazzle
3: translate,fern,underline,yearn,slit,barrage,shin,wanna,barn,rebellion
4: exile,domain,excel,assure,terminate,scandalize,waste,banner,south,deem
5: mouthpiece,ivy,pawn,interpretation,bungalow,craze,anesthetic,donate,barbecue,trailer
